In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD


In [11]:
# Set global parameters
IMG_SIZE = 128
BATCH_SIZE = 32
EPOCHS = 10  # Increased number of epochs for better training
DATA_DIR = "C:/Users/User/OneDrive/pneumonia-detection/chest_xray/chest_xray"
MODEL_SAVE_PATH = "C:/Users/User/OneDrive/pneumonia-detection/best_model.keras"


In [13]:
# Image Preprocessing & Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   rotation_range=40,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   shear_range=0.2,
                                   fill_mode="nearest")

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(os.path.join(DATA_DIR, 'train'),
                                                    target_size=(IMG_SIZE, IMG_SIZE),
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='categorical')

valid_generator = test_datagen.flow_from_directory(os.path.join(DATA_DIR, 'val'),
                                                   target_size=(IMG_SIZE, IMG_SIZE),
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical')

test_generator = test_datagen.flow_from_directory(os.path.join(DATA_DIR, 'test'),
                                                  target_size=(IMG_SIZE, IMG_SIZE),
                                                  batch_size=BATCH_SIZE,
                                                  class_mode='categorical')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [15]:
# Build the VGG19-based Model
base_model = VGG19(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False  # Freezing base model layers

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)  # Reduced layer sizes for optimized performance
x = Dropout(0.3)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [17]:
# Callbacks for monitoring the model's performance
checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', patience=2, verbose=1, factor=0.5, min_lr=0.00001)


In [19]:
# Train the model
history = model.fit(train_generator,
                    validation_data=valid_generator,
                    epochs=EPOCHS,
                    callbacks=[checkpoint, early_stop, reduce_lr])

Epoch 1/10


C:\Users\User\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.6904 - loss: 0.6001
Epoch 1: val_accuracy improved from -inf to 0.87500, saving model to C:/Users/User/OneDrive/pneumonia-detection/best_model.keras
163/163 ━━━━━━━━━━━━━━━━━━━━ 606s 4s/step - accuracy: 0.6907 - loss: 0.5996 - val_accuracy: 0.8750 - val_loss: 0.4579 - learning_rate: 1.0000e-04
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7986 - loss: 0.4365
Epoch 2: val_accuracy did not improve from 0.87500
163/163 ━━━━━━━━━━━━━━━━━━━━ 662s 4s/step - accuracy: 0.7986 - loss: 0.4364 - val_accuracy: 0.7500 - val_loss: 0.5071 - learning_rate: 1.0000e-04
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8047 - loss: 0.4095
Epoch 3: val_accuracy did not improve from 0.87500

Epoch 3: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
163/163 ━━━━━━━━━━━━━━━━━━━━ 554s 3s/step - accuracy: 0.8048 - loss: 0.4095 - val_accuracy: 0.6250 - val_loss: 0.5745 - learning_rate: 1.0000e-04
Epoch 4/10

In [21]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save final model if it's the best one
if not os.path.exists('models/'):
    os.makedirs('models/')
model.save('models/final_model.h5')

20/20 ━━━━━━━━━━━━━━━━━━━━ 56s 3s/step - accuracy: 0.7470 - loss: 0.5579
Test Accuracy: 75.32%
